In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

review sentiment
148  Principally it is the story of two men who wer...  positive
361  Deeply humorous yet honest comedy about a bunc...  positive
91   *** Spoiler in fifth paragraph *** This was an...  positive
850  Do you know that they want to escavate the Moo...  negative
864  All I can really say is that I'm glad that I w...  negative

In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

review sentiment
148  principally story two men part portuguese revo...  positive
361  deeply humorous yet honest comedy bunch grownu...  positive
91   spoiler fifth paragraph amazingly frank uh huh...  positive
850  know want escavate moon real and geneve debate...  negative
864  really say glad knitting sock watching movie w...  negative

In [7]:
df['sentiment'].value_counts()

sentiment
negative    264
positive    236
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

review  sentiment
148  principally story two men part portuguese revo...          1
361  deeply humorous yet honest comedy bunch grownu...          1
91   spoiler fifth paragraph amazingly frank uh huh...          1
850  know want escavate moon real and geneve debate...          0
864  really say glad knitting sock watching movie w...          0

In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/imayank45/Capstone-Project.mlflow')
dagshub.init(repo_owner='imayank45', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Mayank\anaconda3\envs\jarvis\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e82f86f1-1b75-46ba-8bde-5c54f8492028&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=7224580afcd61d097d3242252530f73bf100993a1180d203034fa60ac0a922c2




Accessing as imayank45

Initialized MLflow to track repo "imayank45/Capstone-Project"

Repository imayank45/Capstone-Project initialized!

2025/03/17 00:13:27 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/5fcea33bf02d4d74926d3b5e5e913d2e', creation_time=1742150607805, experiment_id='0', last_update_time=1742150607805, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-17 00:14:44,428 - INFO - Starting MLflow run...
2025-03-17 00:14:45,245 - INFO - Logging preprocessing parameters...
2025-03-17 00:14:46,370 - INFO - Initializing Logistic Regression model...
2025-03-17 00:14:46,372 - INFO - Fitting the model...
2025-03-17 00:14:46,417 - INFO - Model training complete.
2025-03-17 00:14:46,418 - INFO - Logging model parameters...
2025-03-17 00:14:46,799 - INFO - Making predictions...
2025-03-17 00:14:46,800 - INFO - Calculating evaluation metrics...
2025-03-17 00:14:46,815 - INFO - Logging evaluation metrics...
2025-03-17 00:14:48,325 - INFO - Saving and logging the model...
2025/03/17 00:15:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-17 00:15:03,834 - INFO - Model training and logging completed in 18.59 seconds.
2025-03-17 00:15:03,835 - INFO - Accuracy: 0.648
2025-03-17 00:15:03,837 - INFO - Precision: 

🏃 View run bold-cod-874 at: https://dagshub.com/imayank45/Capstone-Project.mlflow/#/experiments/0/runs/e35606039225470a911f57e6452469fe
🧪 View experiment at: https://dagshub.com/imayank45/Capstone-Project.mlflow/#/experiments/0
